In [2]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

import pandas as pd
from shapely.geometry import Polygon

from annotation_utilities import *

[autoreload of utilities2015 failed: Traceback (most recent call last):
  File "/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: __abstractmethods__
]


In [5]:
stack = 'MD589'
# stack = 'MD594'

In [6]:
patch_size = 224
half_size = patch_size/2
stride = 56

dm = DataManager(stack=stack)

w = dm.image_width
h = dm.image_height 

ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                     indexing='xy')
    
sample_locations = np.c_[xs.flat, ys.flat]

# require 30% of the diameter to be within the landmark boundary

margin = int(.3*half_size)

sample_locations_ul = sample_locations - (margin, margin)
sample_locations_ur = sample_locations - (-margin, margin)
sample_locations_ll = sample_locations - (margin, -margin)
sample_locations_lr = sample_locations - (-margin, -margin)

In [7]:
username = 'yuncong'
label_polygons = load_label_polygons_if_exists(stack, username, force=True)

username yuncong does not have any annotations for current section 93 
username yuncong does not have any annotations for current section 94 
username yuncong does not have any annotations for current section 95 
username yuncong does not have any annotations for current section 96 
username yuncong does not have any annotations for current section 97 
username yuncong does not have any annotations for current section 98 
username yuncong does not have any annotations for current section 99 
username yuncong does not have any annotations for current section 100 
username yuncong does not have any annotations for current section 101 
username yuncong does not have any annotations for current section 102 
username yuncong does not have any annotations for current section 103 
username yuncong does not have any annotations for current section 104 
username yuncong does not have any annotations for current section 105 
username yuncong does not have any annotations for current section 106 

In [8]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_surroundIncluded = set(labels[1:] + [l+'_surround' for l in labels[1:]])

annotation_on_sections = get_annotation_on_sections(label_polygons=label_polygons, 
                                                    filtered_labels=label_surroundIncluded)

landmark_range_limits = get_landmark_range_limits(stack=stack, label_polygons=label_polygons, 
                                                  filtered_labels=label_surroundIncluded)

In [9]:
landmark_range_limits

{'12N': [(206, 247)],
 '5N': [(150, 171), (287, 312)],
 '6N': [(205, 213)],
 '7N': [(152, 195), (275, 316)],
 '7n': [(149, 204), (257, 316)],
 'AP': [(215, 233)],
 'Amb': [(164, 170), (295, 299)],
 'LC': [(175, 184), (267, 273)],
 'LRt': [(161, 188), (284, 306)],
 'Pn': [(175, 229), (244, 294)],
 'R': [(187, 215), (238, 245)],
 'RtTg': [(190, 278)],
 'Tz': [(197, 219), (250, 276)],
 'VLL': [(143, 173), (294, 313)],
 'sp5': [(131, 327)]}

In [10]:
from itertools import chain
for l, secs_with_annotations in annotation_on_sections.iteritems():
    secs_within_landmark_range_limits = set(chain(*[range(s1,s2+1) for s1,s2 in landmark_range_limits[l]]))
#     print l, sorted(secs_within_landmark_range_limits - set(secs_with_annotations))
    print l, sorted(secs_within_landmark_range_limits.symmetric_difference(set(secs_with_annotations)))

RtTg []
Tz []
VLL []
7n []
Amb []
sp5 [132, 134, 136, 138, 140, 142, 221]
12N [221]
AP [221]
LRt [285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299]
5N []
6N []
7N []
R []
Pn [245]
LC []


In [14]:
# print 'all annotated classes'
# print label_polygons.keys().tolist()

# labels_to_detect = ['5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 
#                      '6N', 'Amb', 'R', 'Tz', 'Sol', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

# assert set(labels_to_detect).issubset(set(label_polygons.keys()))

In [11]:
labels_to_detect = labels[1:]

In [12]:
section_contains = get_section_contains_labels(label_polygons, filtered_labels=label_surroundIncluded)
secs = section_contains.keys()

In [23]:
progress_bar = FloatProgress(min=np.min(secs), max=np.max(secs))
display(progress_bar, description='identifying landmark patch indices')

indices_allLandmarks_allSections = defaultdict(dict)

# for sec, labs in section_contains.iteritems():
for sec in [160]:
    
#     labs = section_contains[sec]
    labs = ['sp5']

    progress_bar.value = sec
        
    if len(labs) == 0:
        continue

    dm.set_slice(sec)
    mask = dm.load_thumbnail_mask()
    
    indices_fg = np.where(mask[sample_locations[:,1]/32, sample_locations[:,0]/32])[0]
    indices_bg = np.setdiff1d(range(sample_locations.shape[0]), indices_fg)

    indices_inside = {}
    indices_surround = {}
    
    for label in labs:

        path = Path(label_polygons[label][sec])
        indices_inside[label] =  np.where(path.contains_points(sample_locations_ll) &\
                                          path.contains_points(sample_locations_lr) &\
                                          path.contains_points(sample_locations_ul) &\
                                          path.contains_points(sample_locations_ur))[0]
                    
    indices_allInside = np.concatenate(indices_inside.values())    
    
    for label in labs:
 
        surround = Polygon(label_polygons[label][sec]).buffer(500, resolution=2)

        path = Path(list(surround.exterior.coords))
        indices_sur =  np.where(path.contains_points(sample_locations_ll) &\
                                path.contains_points(sample_locations_lr) &\
                                path.contains_points(sample_locations_ul) &\
                                path.contains_points(sample_locations_ur))[0]

        # surround classes do not include patches of any no-surround class
        indices_surround[label] = np.setdiff1d(indices_sur, np.r_[indices_bg, indices_allInside])
        
        
    for l, inds in indices_inside.iteritems():
        indices_allLandmarks_allSections[sec][l] = inds
    for l, inds in indices_surround.iteritems():
        indices_allLandmarks_allSections[sec][l+'_surround'] = inds
    indices_allLandmarks_allSections[sec]['bg'] = indices_bg

indices_allLandmarks_allSections.default_factory = None

In [26]:
41155 in indices_inside

False

In [25]:
41155 in indices_sur

True

In [22]:
41155 in indices_surround['sp5']

True

In [18]:
41155 in indices_allLandmarks_allSections[160]['sp5_surround']

True

In [39]:
# Find which section has sampling locations of a surround class because it is next to an annotated landmark in the z direction.

annotate_z_surround = defaultdict(dict)

num_z_distance = 5

for l, ranges in landmark_range_limits.iteritems():
    
    all_in_range_secs = set(chain(*[range(s1, s2+1) for s1, s2 in ranges]))
    
    for s1, s2 in ranges:
        for before_s1 in set(range(s1 - num_z_distance, s1)) - all_in_range_secs:
            annotate_z_surround[before_s1][l] = s1
        for after_s2 in set(range(s2 + 1, s2 + num_z_distance + 1)) - all_in_range_secs:
            annotate_z_surround[after_s2][l] = s2
            
annotate_z_surround.default_factory = None

In [40]:
# augment the surround sets with above sampling locations

for sec in section_contains:
    if sec in annotate_z_surround:
        for l, annotated_sec in annotate_z_surround[sec].iteritems():
            if l+'_surround' in indices_allLandmarks_allSections[sec]:
                indices_allLandmarks_allSections[sec][l+'_surround'] = \
                np.vstack(indices_allLandmarks_allSections[sec][l+'_surround'],
                          indices_allLandmarks_allSections[annotated_sec][l])
            else:
                indices_allLandmarks_allSections[sec][l+'_surround'] = indices_allLandmarks_allSections[annotated_sec][l]

In [41]:
indices_allLandmarks_allSections_df = pd.DataFrame(indices_allLandmarks_allSections)

In [42]:
indices_allLandmarks_allSections_df

,108,110,112,114,116,118,132,134,136,138,...,298,299,300,301,302,303,304,306,307,308
12N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12N_surround,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"[17586, 17789, 17992, 17993, 18195, 18196, 183...","[18395, 18396, 18397, 18398, 18399, 18400, 184...",NaN,"[18803, 18804, 18805, 19004, 19005, 19006, 190...","[19003, 19205, 19206, 19207, 19408, 19409, 194...",NaN,NaN,NaN,NaN,NaN
5N_surround,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"[15961, 15962, 15963, 16162, 16163, 16164, 161...","[16566, 16567, 16568, 16569, 16570, 16571, 165...",NaN,"[16974, 16975, 16976, 16977, 16978, 17175, 171...","[17173, 17174, 17175, 17176, 17375, 17376, 173...","[19003, 19205, 19206, 19207, 19408, 19409, 194...","[19003, 19205, 19206, 19207, 19408, 19409, 194...","[19003, 19205, 19206, 19207, 19408, 19409, 194...","[19003, 19205, 19206, 19207, 19408, 19409, 194...",NaN
6N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6N_surround,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"[26351, 26352, 26353, 26553, 26554, 26555, 265...","[26351, 26352, 26553, 26554, 26555, 26556, 265...","[26753, 26754, 26755, 26756, 26954, 26955, 269...","[26551, 26552, 26753, 26754, 26755, 26756, 269...","[26957, 26958, 27160, 27161, 27362, 27363, 273...","[27969, 28172, 28173, 28174, 28175, 28375, 283...","[27567, 27769, 27770, 27971, 27972, 27973, 281...",NaN,NaN,NaN
7N_surround,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"[24323, 24522, 24523, 24524, 24525, 24526, 245...","[24526, 24723, 24724, 24725, 24726, 24727, 247...","[24924, 24925, 24926, 24927, 24928, 24929, 249...","[24722, 24723, 24724, 24725, 24726, 24727, 249...","[25130, 25331, 25332, 25333, 25334, 25335, 253...","[25938, 25939, 26138, 26139, 26140, 26141, 261...","[25739, 25740, 25741, 25742, 25743, 25939, 259...","[27567, 27769, 27770, 27971, 27972, 27973, 281...","[27567, 27769, 27770, 27971, 27972, 27973, 281...","[27567, 27769, 27770, 27971, 27972, 27973, 281..."
7n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"[21269, 21471, 21472, 21673, 21674, 21675, 216...","[21067, 21269, 21270, 21471, 21472, 21473, 216...","[20864, 20865, 21066, 21067, 21068, 21269, 212...","[20865, 21067, 21068, 21270, 21271, 21473, 214...","[21068, 21270, 21271, 21472, 21473, 21474, 214...","[20662, 20865, 21067, 21068, 21069, 21270, 212...","[21676, 21677, 21878, 21879, 21880, 22081, 220...",NaN,NaN,NaN
7n_surround,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"[19442, 19443, 19444, 19641, 19642, 19643, 196...","[19439, 19440, 19441, 19442, 19443, 19444, 194...","[19039, 19040, 19041, 19235, 19236, 19237, 192...","[19038, 19039, 19040, 19237, 19238, 19239, 192...","[19037, 19038, 19238, 19239, 19240, 19241, 192...","[18838, 18839, 19034, 19035, 19036, 19037, 190...","[19645, 19646, 19647, 19648, 19846, 19847, 198...","[21676, 21677, 21878, 21879, 21880, 22081, 220...","[21676, 21677, 21878, 21879, 21880, 22081, 220...","[21676, 21677, 21878, 21879, 21880, 22081, 220..."


In [43]:
patch_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/'
table_filepath = os.path.join(patch_dir, '%(stack)s_indices_allLandmarks_allSections.h5'%{'stack':stack})
indices_allLandmarks_allSections_df.to_hdf(table_filepath, 'indices_allLandmarks_allSections')
pd.Series([patch_size, stride, w, h]).to_hdf(table_filepath, 'grid_parameters')

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->[108, 110, 112, 114, 116, 118, 132, 134, 136, 138, 140, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 254, 256, 257, 258, 259, 260, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 2

In [44]:
# sync with workstation

workstation_patch_dir = '/home/yuncong/CSHL_data_patches'
cmd = 'scp %(fn)s yuncong@132.239.73.151:%(dest)s' % {'fn': table_filepath, 'dest': workstation_patch_dir}
os.system(cmd)

0